In [49]:
from inspect import getfullargspec
from typing import Callable

import numpy as np

from sfeprapy.func.csv import csv_to_list_of_dicts, dict_of_ndarray_to_csv
from sfeprapy.func.xlsx import xlsx_to_dict
from sfeprapy.mcs0.calcs import (
    decide_fire, evaluate_fire_temperature, solve_protection_thickness, solve_time_equivalence_iso834
)

In [30]:
fp_input = r'C:\Users\ian\Desktop\sfeprapy_test\test.xlsx'
data = xlsx_to_dict(fp_input)

In [31]:
data

{'CASE_1': {'case_name': 'CASE_1',
  'n_simulations': 2500,
  'fire_time_step': 10,
  'fire_time_duration': 18000,
  'fire_hrr_density:dist': 'uniform_',
  'fire_hrr_density:lbound': 0.249,
  'fire_hrr_density:ubound': 0.251,
  'fire_load_density:dist': 'gumbel_r_',
  'fire_load_density:lbound': 10,
  'fire_load_density:ubound': 1500,
  'fire_load_density:mean': 420,
  'fire_load_density:sd': 126,
  'fire_spread_speed:dist': 'uniform_',
  'fire_spread_speed:lbound': 0.0035,
  'fire_spread_speed:ubound': 0.019,
  'fire_nft_limit:dist': 'norm_',
  'fire_nft_limit:lbound': 623.15,
  'fire_nft_limit:ubound': 1473.15,
  'fire_nft_limit:mean': 1323.15,
  'fire_nft_limit:sd': 93,
  'fire_combustion_efficiency:dist': 'uniform_',
  'fire_combustion_efficiency:lbound': 0.8,
  'fire_combustion_efficiency:ubound': 1,
  'window_open_fraction:dist': 'lognorm_mod_',
  'window_open_fraction:ubound': 0.9999,
  'window_open_fraction:lbound': 0.0001,
  'window_open_fraction:mean': 0.2,
  'window_open_fra

In [32]:
fp_output = r'C:\Users\ian\Desktop\sfeprapy_test\test\CASE_1.csv'
data_out = csv_to_list_of_dicts(fp_output)
data_out = [{k: float(v) for k, v in i.items()} for i in data_out]

In [33]:
print(data_out[0])

{'index': 0.0, 'beam_position_horizontal': 25.0137, 'fire_combustion_efficiency': 0.932507, 'fire_hrr_density': 0.249988, 'fire_nft_limit': 1457.04, 'fire_spread_speed': 0.0108317, 'window_open_fraction': 0.319879, 'fire_load_density': 385.994, 'fire_type': 1.0, 't1': 1439.84, 't2': 2885.06, 't3': 4324.89, 'solver_steel_temperature_solved': 822.653, 'solver_time_critical_temp_solved': 4010.0, 'solver_protection_thickness': 0.0141897, 'solver_iter_count': 8.0, 'solver_time_equivalence_solved': 3419.79, 'timber_charring_rate': 0.0, 'timber_exposed_duration': 0.0, 'timber_solver_iter_count': 0.0, 'timber_fire_load': nan, 'timber_charred_depth': nan, 'timber_charred_mass': nan, 'timber_charred_volume': nan}


In [45]:
def teq_main(
        index: int, beam_cross_section_area: float, beam_position_vertical: float, beam_position_horizontal: float,
        beam_rho: float, fire_time_duration: float, fire_time_step: float, fire_combustion_efficiency: float,
        fire_gamma_fi_q: float, fire_hrr_density: float, fire_load_density: float, fire_mode: int,
        fire_nft_limit: float, fire_spread_speed: float, fire_t_alpha: float, fire_tlim: float, protection_c: float,
        protection_k: float, protection_protected_perimeter: float, protection_rho: float, room_breadth: float,
        room_depth: float, room_height: float, room_wall_thermal_inertia: float, solver_temperature_goal: float,
        solver_max_iter: int, solver_thickness_lbound: float, solver_thickness_ubound: float, solver_tol: float,
        window_height: float, window_open_fraction: float, window_width: float, window_open_fraction_permanent: float,
        phi_teq: float = 1.0, timber_exposed_area: float = 0., timber_charred_depth=None, timber_charring_rate=None,
        timber_hc: float = None, timber_density: float = None, timber_depth: float = None,
        timber_solver_tol: float = None, timber_solver_ilim: float = None, occupancy_type: str = None,
        car_cluster_size: int = None,
) -> tuple:
    # Make the longest dimension between (room_depth, room_breadth) as room_depth
    if room_depth < room_breadth:
        room_depth += room_breadth
        room_breadth = room_depth - room_breadth
        room_depth -= room_breadth

    # todo: wip for car park!!!
    if occupancy_type == '__CAR_PARK__':
        fire_mode = 1  # force to travelling fire only
        # work out new room_depth_car based on how many cars are involved in fire
        if car_cluster_size is not None and car_cluster_size >= 0:
            car_cluster_size = int(car_cluster_size) + 1
            room_depth_original = float(room_depth)
            parking_bay_width = 2.3
            n_parking_bay_row = 2
            average_area_per_parking_bay = 4283 / 202

            room_depth = car_cluster_size * parking_bay_width / n_parking_bay_row
            room_floor_area = car_cluster_size * average_area_per_parking_bay
            room_breadth = room_floor_area / room_depth

            beam_position_horizontal = (beam_position_horizontal / room_depth_original) * room_depth

    window_open_fraction = (
            window_open_fraction * (1 - window_open_fraction_permanent) + window_open_fraction_permanent
    )

    # Fix ventilation opening size, so it doesn't exceed wall area
    if window_height > room_height:
        window_height = room_height

    # Calculate fire time, this is used for all fire curves in the calculation
    fire_time = np.arange(0, fire_time_duration + fire_time_step, fire_time_step, dtype=float)

    # Calculate ISO 834 fire temperature
    # fire_time_iso834 = fire_time
    # fire_temperature_iso834 = (345.0 * np.log10((fire_time / 60.0) * 8.0 + 1.0) + 20.0) + 273.15  # in [K]

    # initialise solver iteration count for timber fuel contribution
    timber_solver_iter_count = -1
    timber_exposed_duration = 0  # initial condition, timber exposed duration
    _fire_load_density_ = float(fire_load_density)  # preserve original fire load density

    while True:
        timber_solver_iter_count += 1
        # the following `if` decide whether to calculate `timber_charred_depth_i` from `timber_charring_rate` or
        if (
                timber_exposed_area is not None and
                timber_exposed_area > 0 and
                (timber_charred_depth is not None or timber_charring_rate is not None)
        ):
            if timber_charred_depth is None:
                # calculate from timber charring rate
                if isinstance(timber_charring_rate, (float, int)):
                    timber_charring_rate_i = timber_charring_rate
                elif isinstance(timber_charring_rate, Callable):
                    timber_charring_rate_i = timber_charring_rate(timber_exposed_duration)
                else:
                    raise TypeError('`timber_charring_rate_i` is not numerical nor Callable type')
                timber_charring_rate_i *= 1. / 1000.  # [mm/min] -> [m/min]
                timber_charring_rate_i *= 1. / 60.  # [m/min] -> [m/s]
                timber_charred_depth_i = timber_charring_rate_i * timber_exposed_duration
            else:
                # calculate from timber charred depth
                if isinstance(timber_charred_depth, (float, int)):
                    timber_charred_depth_i = timber_charred_depth
                elif isinstance(timber_charred_depth, Callable):
                    timber_charred_depth_i = timber_charred_depth(timber_exposed_duration)
                else:
                    raise TypeError('`timber_charring_rate_i` is not numerical nor Callable type')
                timber_charred_depth_i /= 1000.

            # make sure the calculated charred depth does not exceed the available timber depth
            if timber_depth is not None:
                timber_charred_depth_i = min(timber_charred_depth_i, timber_depth)

            timber_charred_volume = timber_charred_depth_i * timber_exposed_area
            timber_charred_mass = timber_density * timber_charred_volume
            timber_fire_load = timber_charred_mass * timber_hc
            timber_fire_load_density = timber_fire_load / (room_breadth * room_depth)
        else:
            timber_charred_volume = np.nan
            timber_charred_depth_i = np.nan
            timber_charred_mass = np.nan
            timber_fire_load = np.nan
            timber_fire_load_density = np.nan

        if np.isnan(timber_fire_load_density):
            fire_load_density = _fire_load_density_
        else:
            fire_load_density = _fire_load_density_ + timber_fire_load_density

        # To check what design fire to use
        fire_type = decide_fire(
            window_height=window_height, window_width=window_width, window_open_fraction=window_open_fraction,
            room_breadth=room_breadth, room_depth=room_depth, room_height=room_height, fire_mode=fire_mode,
            fire_load_density=fire_load_density, fire_combustion_efficiency=fire_combustion_efficiency,
            fire_hrr_density=fire_hrr_density, fire_spread_speed=fire_spread_speed
        )

        # To calculate design fire temperature
        fire_temperature, beam_position_horizontal, t1, t2, t3 = evaluate_fire_temperature(
            window_height=window_height, window_width=window_width, window_open_fraction=window_open_fraction,
            room_breadth=room_breadth, room_depth=room_depth, room_height=room_height,
            room_wall_thermal_inertia=room_wall_thermal_inertia, fire_tlim=fire_tlim, fire_type=fire_type,
            fire_time=fire_time, fire_nft_limit=fire_nft_limit, fire_load_density=fire_load_density,
            fire_combustion_efficiency=fire_combustion_efficiency, fire_hrr_density=fire_hrr_density,
            fire_spread_speed=fire_spread_speed, fire_t_alpha=fire_t_alpha, fire_gamma_fi_q=fire_gamma_fi_q,
            beam_position_vertical=beam_position_vertical, beam_position_horizontal=beam_position_horizontal
        )

        # To solve protection thickness at critical temperature
        # inputs.update(solve_protection_thickness(**inputs))
        (
            solver_steel_temperature_solved, solver_time_critical_temp_solved, solver_protection_thickness,
            solver_iter_count
        ) = solve_protection_thickness(
            fire_time=fire_time, fire_temperature=fire_temperature, beam_cross_section_area=beam_cross_section_area,
            beam_rho=beam_rho, protection_k=protection_k, protection_rho=protection_rho, protection_c=protection_c,
            protection_protected_perimeter=protection_protected_perimeter,
            solver_temperature_goal=solver_temperature_goal, solver_max_iter=solver_max_iter,
            solver_thickness_ubound=solver_thickness_ubound, solver_thickness_lbound=solver_thickness_lbound,
            solver_tol=solver_tol
        )

        # To solve time equivalence in ISO 834
        solver_time_equivalence_solved = solve_time_equivalence_iso834(
            fire_time=fire_time, beam_cross_section_area=beam_cross_section_area, beam_rho=beam_rho,
            protection_k=protection_k, protection_rho=protection_rho, protection_c=protection_c,
            protection_protected_perimeter=protection_protected_perimeter,
            solver_temperature_goal=solver_temperature_goal, solver_protection_thickness=solver_protection_thickness,
            phi_teq=phi_teq
        )

        # additional fuel contribution from timber
        if timber_exposed_area <= 0 or timber_exposed_area is None:  # no timber exposed
            # Exit timber fuel contribution solver if:
            #     1. no timber exposed
            #     2. timber exposed area undefined
            break
        elif timber_solver_iter_count >= timber_solver_ilim:
            solver_convergence_status = np.nan
            solver_time_critical_temp_solved = np.nan
            solver_time_equivalence_solved = np.nan
            solver_steel_temperature_solved = np.nan
            solver_protection_thickness = np.nan
            solver_iter_count = np.nan
            timber_exposed_duration = np.nan
            break
        elif not -np.inf < solver_protection_thickness < np.inf:
            # no protection thickness solution
            timber_exposed_duration = solver_protection_thickness
            break
        elif abs(timber_exposed_duration - solver_time_equivalence_solved) <= timber_solver_tol:
            # convergence sought successfully
            break
        else:
            timber_exposed_duration = solver_time_equivalence_solved

    timber_charring_rate = timber_charred_depth_i / timber_exposed_duration if timber_exposed_duration else 0
    timber_exposed_duration = timber_exposed_duration
    timber_solver_iter_count = timber_solver_iter_count
    timber_fire_load = timber_fire_load
    timber_charred_depth = timber_charred_depth_i

    return (
        fire_time, fire_temperature, solver_time_equivalence_solved, index, fire_type, t1, t2, t3,
        solver_steel_temperature_solved, solver_time_critical_temp_solved, solver_protection_thickness,
        solver_iter_count, solver_time_equivalence_solved, timber_charring_rate, timber_exposed_duration,
        timber_solver_iter_count, timber_fire_load, timber_charred_depth, timber_charred_mass, timber_charred_volume,
    )


In [35]:
_ = getfullargspec(teq_main)
keys = tuple(_.args)
defaults = _.defaults

In [36]:
indices = (990, 991, 992)

In [50]:
fp_save_fires = r'C:\Users\ian\Desktop\sfeprapy_test\test\CASE_1.fires.csv'
data_fires = dict()
for i in data_out:
    if i['index'] in indices:
        kwargs_all = {**i, **data['CASE_1']}
        kwargs = {k: v for k, v in kwargs_all.items() if k in keys}
        t, T, t_eq, *_ = teq_main(**kwargs)
        if 'time' not in data_fires.keys():
            data_fires['time'] = t
        data_fires[f'{i["index"]}'] = T

dict_of_ndarray_to_csv(fp_save_fires, data_fires)